# HDT 8 
- Mariana David
- Alejandra Guzmán
- Jorge Caballeros 20009
 

1. Use los mismos conjuntos de entrenamiento y prueba que utilizó en las hojas anteriores.


2. Seleccione como variable respuesta la que creó con las categorías del precio de la casa.

Importing libraries

In [50]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from sklearn import tree
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import confusion_matrix, accuracy_score
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import learning_curve

3. Genere dos modelos de redes neuronales que sean capaz de clasificar usando la variable 
respuesta que categoriza las casas en baratas, medias y caras. Estos modelos deben tener 
diferentes topologías y funciones de activación.

In [51]:
# Load data
data = pd.read_csv('train.csv')
data = pd.DataFrame(data)
data.describe()
# Preprocessing
# Elimina columnas que no son relevantes o que contienen muchos valores faltantes
data = data.drop(['Id', 'Alley', 'PoolQC', 'Fence', 'MiscFeature'], axis=1)

# Elimina filas que tienen valores faltantes en columnas importantes
data = data.dropna(subset=['SalePrice', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'LotFrontage', 'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical'])

# Rellena los valores faltantes con la media o la moda de la columna
data['LotFrontage'] = data['LotFrontage'].fillna(data['LotFrontage'].mean())
data['MasVnrType'] = data['MasVnrType'].fillna(data['MasVnrType'].mode()[0])
data['MasVnrArea'] = data['MasVnrArea'].fillna(data['MasVnrArea'].mean())
data['BsmtQual'] = data['BsmtQual'].fillna(data['BsmtQual'].mode()[0])
data['BsmtCond'] = data['BsmtCond'].fillna(data['BsmtCond'].mode()[0])
data['BsmtExposure'] = data['BsmtExposure'].fillna(data['BsmtExposure'].mode()[0])
data['BsmtFinType1'] = data['BsmtFinType1'].fillna(data['BsmtFinType1'].mode()[0])
data['BsmtFinType2'] = data['BsmtFinType2'].fillna(data['BsmtFinType2'].mode()[0])
data['Electrical'] = data['Electrical'].fillna(data['Electrical'].mode()[0])

# Convierte las variables categóricas en variables numéricas
data = pd.get_dummies(data)

# Guarda los datos limpios en un nuevo archivo CSV
data.to_csv('train_cleaned.csv', index=False)



4. Use los modelos para predecir el valor de la variable respuesta

5. Haga las matrices de confusión respectivas.

6. Compare los resultados obtenidos con los diferentes modelos de clasificación usando redes 
neuronales en cuanto a efectividad, tiempo de procesamiento y equivocaciones (donde el 
algoritmo se equivocó más, donde se equivocó menos y la importancia que tienen los 
errores)

7. Analice si no hay sobreajuste en los modelos. Use para esto la curva de aprendizaje.

8. Para el modelo elegido de clasificación tunee los parámetros y discuta si puede mejorar 
todavía el modelo sin llegar a sobre ajustarlo.

9. Seleccione ahora el SalesPrice como variable respuesta.

10. Genere dos modelos de regresión con redes neuronales con diferentes topologías y 
funciones de activación para predecir el precio de las casas.

11. Compare los dos modelos de regresión y determine cuál funcionó mejor para predecir el 
precio de las casas

12. Analice si no hay sobreajuste en los modelos. Use para esto la curva de aprendizaje.



13. Para el modelo elegido de regresión tunee los parámetros y discuta si puede mejorar 
todavía el modelo sin llegar a sobre ajustarlo. 


14. Compare la eficiencia del mejor modelo de RNA con los resultados obtenidos con los 
algoritmos de las hojas de trabajo anteriores. ¿Cuál es mejor para predecir? ¿Cuál se 
demoró más en procesar?


15. Compare los resultados del mejor modelo de esta hoja para clasificar con los resultados de 
los algoritmos usados para clasificar de las hojas de trabajo anteriores


16. Compare los resultados del mejor modelo para predecir el precio de venta con los 
resultados de los algoritmos usados para el mismo propósito de las hojas de trabajo 
anteriores.


17. Ahora que ha usado todos los modelos que hemos visto y aplicados al conjunto de datos 
llegue a conclusiones sobre cual es o cuales son los mejores modelos para clasificar dadas 
las características del conjunto de datos. ¿Cuál o cuáles son los mejores para predecir el 
precio de las casas? Una tabla de resumen con las métricas de los modelos le puede resultar 
muy útil para esto.


18. Genere un informe de los resultados y las explicaciones.